This is a companion notebook for the book [Deep Learning with Python, Third Edition](https://www.manning.com/books/deep-learning-with-python-third-edition). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

The book's contents are available online at [deeplearningwithpython.io](https://deeplearningwithpython.io).

In [ ]:
!pip install keras keras-hub --upgrade -q

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [ ]:
# @title
import os
from IPython.core.magic import register_cell_magic

@register_cell_magic
def backend(line, cell):
    current, required = os.environ.get("KERAS_BACKEND", ""), line.split()[-1]
    if current == required:
        get_ipython().run_cell(cell)
    else:
        print(
            f"This cell requires the {required} backend. To run it, change KERAS_BACKEND to "
            f"\"{required}\" at the top of the notebook, restart the runtime, and rerun the notebook."
        )

## Classification and regression

이 장에서는 다음 내용을 다룹니다.* 실제 머신러닝 워크플로의 첫 번째 예시
* 이진 및 범주형 분류 문제 처리
* 연속형 회귀 문제 처리 학습 이 장에서는 신경망을 사용하여 실제 문제를 해결하는 방법을 배우게 됩니다. 2장과 3장에서 배운 내용을 다지고, 신경망의 가장 일반적인 세 ​​가지 활용 사례인 이진 분류, 범주형 분류, 스칼라 회귀에 적용해 봅니다.

* 영화 리뷰를 긍정적 또는 부정적으로 분류하기(이진 분류)
* 뉴스 기사를 주제별로 분류하기(범주형 분류)
* 부동산 데이터를 기반으로 주택 가격을 예측하기(스칼라 회귀)

이러한 예제들을 통해 데이터 전처리, 기본적인 모델 아키텍처 원리, 모델 평가 등 머신러닝 워크플로의 전반적인 과정을 처음 접하게 될 것입니다.

이 장을 마치면 벡터 데이터에 대한 간단한 분류 및 회귀 문제를 신경망을 사용하여 해결할 수 있게 됩니다. 그리고 5장에서는 머신러닝에 대한 보다 심도 있고 이론적인 이해를 쌓아 나갈 준비가 될 것입니다. 살펴볼 준비가 될 것입니다.

**분류 및 회귀 용어집**

분류와 회귀에는 많은 전문 용어가 있습니다. 앞선 예제에서 이미 몇 가지 용어를 접했고, 앞으로의 장에서도 더 많이 보게 될 것입니다. 이러한 용어들은 머신러닝에 특화된 정확한 정의를 가지고 있으므로, 숙지해 두는 것이 중요합니다.

* 샘플 또는 입력 — 모델에 입력되는 하나의 데이터 포인트입니다.
* 예측 또는 출력 — 모델에서 나오는 결과입니다.
* 목표 — 실제 값입니다. 외부 데이터에 따르면 모델이 이상적으로 예측해야 하는 값입니다.
* 예측 오류 또는 손실 값 — 모델의 예측과 목표 사이의 거리를 나타내는 척도입니다.
* 클래스 — 분류 문제에서 선택할 수 있는 가능한 레이블 집합입니다. 예를 들어, 고양이와 강아지 사진을 분류할 때 "강아지"와 "고양이"가 두 가지 클래스입니다.
* 레이블 — 분류 문제에서 클래스 주석의 특정 인스턴스입니다. 예를 들어, 사진 #1234가 "강아지" 클래스로 주석 처리되었다면, "강아지"는 사진 #1234의 레이블입니다.

* 정답 데이터 또는 주석 — 데이터 세트의 모든 목표값으로, 일반적으로 사람이 수집합니다.
* 이진 분류 — 각 입력 샘플을 두 개의 배타적인 범주로 분류해야 하는 분류 작업입니다.
범주형 분류 또는 다중 클래스 분류 — 각 입력 샘플을 두 개 이상의 범주로 분류해야 하는 분류 작업입니다. 예를 들어, 손으로 쓴 숫자를 분류하는 경우가 있습니다.
* 다중 레이블 분류 — 각 입력 샘플에 여러 레이블을 할당할 수 있는 분류 작업입니다. 예를 들어, 주어진 이미지에 고양이와 개가 모두 포함되어 있고 "고양이" 레이블과 "개" 레이블로 주석을 달아야 하는 경우입니다. 이미지당 레이블 수는 일반적으로 가변적입니다.
* 스칼라 회귀 — 목표값이 연속적인 스칼라 값인 작업입니다. 주택 가격 예측이 좋은 예입니다. 다양한 목표 가격은 연속적인 공간을 형성합니다.
* 벡터 회귀 — 목표값이 연속적인 값들의 집합인 작업입니다. 예를 들어, 연속적인 벡터를 예측하는 경우가 있습니다. 이미지에서 경계 상자의 좌표와 같은 여러 값에 대해 회귀를 수행하는 경우 벡터 회귀를 수행하는 것입니다.
* 미니 배치 또는 배치 — 모델이 동시에 처리하는 작은 샘플 집합(일반적으로 8~128개)입니다. 샘플 수는 GPU에서 메모리 할당을 용이하게 하기 위해 종종 2의 거듭제곱입니다. 학습 시 미니 배치는 모델 가중치에 적용되는 단일 경사 하강 업데이트를 계산하는 데 사용됩니다.

### Classifying movie reviews: A binary classification example

이진 분류(이진 분류)는 머신러닝 문제 유형 중 가장 흔한 유형 중 하나입니다. 이 예제에서는 영화 리뷰의 텍스트 내용을 기반으로 리뷰를 긍정적 또는 부정적으로 분류하는 방법을 배우게 됩니다.

#### The IMDb dataset

여러분은 인터넷 영화 데이터베이스(IMDb)에서 가져온 5만 개의 극단적으로 양극화된 리뷰로 구성된 IMDb 데이터셋을 사용하게 됩니다. 이 데이터셋은 학습용 2만 5천 개와 테스트용 2만 5천 개로 나뉘며, 각 데이터셋은 긍정적인 리뷰와 부정적인 리뷰가 각각 50%씩 포함되어 있습니다.

MNIST 데이터셋과 마찬가지로 IMDb 데이터셋도 Keras와 함께 제공됩니다. 데이터셋은 이미 전처리되어 있습니다. 리뷰(단어 시퀀스)는 각 정수가 사전의 특정 단어를 나타내는 정수 시퀀스로 변환되어 있습니다. 따라서 모델 구축, 학습 및 평가에 집중할 수 있습니다. 14장에서는 원시 텍스트 입력을 처음부터 처리하는 방법을 배우게 됩니다.

다음 코드는 데이터셋을 로드합니다(처음 실행할 때 약 80MB의 데이터가 컴퓨터에 다운로드됩니다).

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000
)

`num_words=10000` 인수는 학습 데이터에서 가장 빈번하게 나타나는 단어 10,000개만 유지한다는 의미입니다. 드물게 나타나는 단어는 제거됩니다. 이렇게 하면 관리 가능한 크기의 벡터 데이터를 사용할 수 있습니다. 이 제한을 설정하지 않으면 학습 데이터에 88,585개의 고유 단어가 포함되어 불필요하게 많은 양의 단어를 사용하게 됩니다. 이러한 단어 중 상당수는 단일 샘플에서만 나타나므로 분류에 의미 있게 사용할 수 없습니다.

변수 `train_data`와 `test_data`는 리뷰의 NumPy 배열입니다. 각 리뷰는 단어 인덱스(단어 시퀀스를 인코딩) 목록입니다. `train_labels`와 `test_labels`는 0과 1로 이루어진 NumPy 배열이며, 0은 부정적, 1은 긍정적을 나타냅니다.

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

가장 자주 사용되는 단어 10,000개로 제한했기 때문에 단어 인덱스는 10,000을 초과하지 않습니다.

In [ ]:
max([max(sequence) for sequence in train_data])

재미 삼아 이 리뷰 중 하나를 영어 단어로 빠르게 해독해 봅시다.

In [ ]:
# word_index is a dictionary mapping words to an integer index.
word_index = imdb.get_word_index()
# Reverses it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Decodes the review. Note that the indices are offset by 3 because 0,
# 1, and 2 are reserved indices for "padding," "start of sequence," and
# "unknown."
decoded_review = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]]
)

우리가 얻은 결과를 살펴보겠습니다.

In [ ]:
decoded_review[:100]

앞에 붙은 물음표(?)는 각 리뷰 앞에 추가된 시작 토큰임을 참고하세요.

#### Preparing the data

정수 리스트를 신경망에 직접 입력할 수는 없습니다. 리스트는 길이가 모두 다르기 때문입니다. 신경망은 연속적인 데이터 배치를 처리하도록 설계되어 있습니다. 따라서 리스트를 텐서로 변환해야 합니다. 변환 방법은 두 가지가 있습니다.

첫 번째는 리스트의 길이를 모두 동일하게 패딩한 다음, (샘플 수, 최대 길이) 형태의 정수 텐서로 변환하고, 이러한 정수 텐서를 처리할 수 있는 레이어(이 책 후반부에서 자세히 다룰 임베딩 레이어)로 모델을 시작하는 것입니다.

두 번째는 리스트를 멀티핫 인코딩하여 모든 가능한 단어의 존재 여부를 나타내는 0과 1로 이루어진 벡터로 변환하는 것입니다. 예를 들어, 시퀀스 [8, 5]는 인덱스 5와 8을 제외한 모든 요소가 0이고, 5와 8만 1인 10,000차원 벡터로 변환됩니다.
여기서는 두 번째 방법을 사용하여 데이터를 벡터화하겠습니다. 수동으로 벡터화하는 과정은 다음과 같습니다.

In [ ]:
import numpy as np

def multi_hot_encode(sequences, num_classes):
    # Creates an all-zero matrix of shape (len(sequences), num_classes)
    results = np.zeros((len(sequences), num_classes))
    for i, sequence in enumerate(sequences):
        # Sets specific indices of results[i] to 1s
        results[i][sequence] = 1.0
    return results

# Vectorized training data
x_train = multi_hot_encode(train_data, num_classes=10000)
# Vectorized test data
x_test = multi_hot_encode(test_data, num_classes=10000)

현재 샘플의 모습은 다음과 같습니다.

In [ ]:
x_train[0]

입력 시퀀스를 벡터화하는 것 외에도 레이블도 벡터화해야 하는데, 이는 간단합니다. 레이블은 이미 NumPy 배열이므로 정수형을 부동소수점형으로 변환하기만 하면 됩니다.

In [ ]:
y_train = train_labels.astype("float32")
y_test = test_labels.astype("float32")

이제 데이터를 신경망에 입력할 준비가 되었습니다.

#### Building your model

입력 데이터는 벡터이고 레이블은 스칼라(1과 0)입니다. 이는 여러분이 접하게 될 가장 간단한 문제 유형 중 하나입니다. 이러한 문제에서 좋은 성능을 보이는 모델 유형은 ReLU 활성화 함수를 사용하는 밀집 연결(Dense) 레이어의 스택입니다.

이러한 Dense 레이어 스택을 설계할 때 두 가지 중요한 아키텍처 결정 사항이 있습니다.

* 레이어 개수
* 각 레이어의 유닛 개수

5장에서는 이러한 선택을 안내하는 정형 원리를 배우게 됩니다. 지금은 다음과 같은 아키텍처를 사용하겠습니다.

* 각각 16개의 유닛을 가진 두 개의 중간 레이어
* 현재 리뷰의 감정에 대한 스칼라 예측값을 출력하는 세 번째 레이어

그림 4.1은 모델의 구조를 보여줍니다. 다음은 이전에 살펴본 MNIST 예제와 유사한 Keras 구현입니다.

In [ ]:
import keras
from keras import layers

model = keras.Sequential(
    [
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

<img src="https://deeplearningwithpython.io/images/ch04/3_layer_network.cf1b1cd7.png" width="200"><p style="text-align:center">Figure 4.1: The three-layer model</p>

각 Dense 레이어에 전달되는 첫 번째 인수는 레이어의 유닛 수, 즉 레이어의 표현 공간 차원입니다. 2장과 3장에서 배웠듯이, ReLU 활성화 함수를 사용하는 각 Dense 레이어는 다음과 같은 텐서 연산 과정을 구현합니다.
```
output = relu(dot(input, W) + b)
```
16개의 유닛을 사용한다는 것은 가중치 행렬 W의 형태가 (입력 차원, 16)이 된다는 것을 의미합니다. W와의 내적은 입력 데이터를 16차원 표현 공간으로 투영합니다(그리고 나서 편향 벡터 b를 더하고 ReLU 활성화 함수를 적용합니다). 표현 공간의 차원은 "모델이 내부 표현을 학습할 때 허용하는 자유도"로 직관적으로 이해할 수 있습니다. 유닛 수가 많을수록(표현 공간의 차원이 높을수록) 모델은 더 복잡한 표현을 학습할 수 있지만, 계산 비용이 증가하고 원치 않는 패턴(훈련 데이터에서는 성능을 향상시키지만 테스트 데이터에서는 그렇지 않은 패턴)을 학습할 가능성이 높아집니다.

중간 레이어는 ReLU 활성화 함수를 사용하고, 최종 레이어는 시그모이드 활성화 함수를 사용하여 확률(0에서 1 사이의 점수로, 리뷰가 긍정적일 가능성을 나타냄)을 출력합니다. ReLU(정류 선형 단위) 함수는 음수 값을 0으로 만드는 함수인 반면(그림 4.2 참조), 시그모이드 함수는 임의의 값을 [0, 1] 구간으로 "압축"하여 확률로 해석될 수 있는 값을 출력합니다(그림 4.3 참조).

<img src="https://deeplearningwithpython.io/images/ch04/The-rectified-linear-unit-function.351095bf.png" width="300"><p style="text-align:center">Figure 4.2: The rectified linear unit function</p>

<img src="https://deeplearningwithpython.io/images/ch04/The-sigmoid-function.eac1368d.png" width="300"><p style="text-align:center">Figure 4.3: The sigmoid function</p>

**활성화 함수란 무엇이며, 왜 필요한가요?**

ReLU와 같은 활성화 함수(비선형 함수라고도 함)가 없다면, Dense 레이어는 두 개의 선형 연산, 즉 내적과 덧셈으로만 구성됩니다.

```
output = dot(input, W) + b
```
따라서 이 레이어는 입력 데이터의 선형 변환(아핀 변환)만 학습할 수 있습니다. 레이어의 가설 공간은 입력 데이터를 16차원 공간으로 변환하는 모든 가능한 선형 변환의 집합이 됩니다. 이러한 가설 공간은 너무 제한적이어서 여러 레이어를 사용해도 선형 연산만 수행하게 되므로, 여러 레이어를 추가해도 가설 공간이 확장되지 않습니다(2장에서 살펴본 바와 같이).

깊은 표현을 통해 훨씬 풍부한 가설 공간을 활용하려면 비선형 함수 또는 활성화 함수가 필요합니다. ReLU는 딥러닝에서 가장 널리 사용되는 활성화 함수이지만, PreLU, ELU 등 비슷한 이름을 가진 다른 활성화 함수들도 많이 있습니다.

마지막으로 손실 함수와 옵티마이저를 선택해야 합니다. 이진 분류 문제이고 모델의 출력이 확률이기 때문에(모델의 마지막 레이어는 시그모이드 활성화 함수를 사용하는 단일 유닛 레이어입니다), `binary_crossentropy` 손실 함수를 사용하는 것이 가장 좋습니다. 물론 다른 선택지도 있습니다. 예를 들어 `mean_squared_error`를 사용할 수도 있습니다. 하지만 확률을 출력하는 모델을 다룰 때는 일반적으로 `crossentropy`가 최적의 선택입니다. `crossentropy`는 정보 이론 분야의 개념으로, 확률 분포 간의 거리, 즉 실제 분포와 예측값 사이의 거리를 측정합니다.

옵티마이저는 거의 모든 문제에서 좋은 기본 선택으로 여겨지는 `adam`을 사용하겠습니다.

다음 단계에서는 `adam` 옵티마이저와 `binary_crossentropy` 손실 함수를 사용하여 모델을 구성합니다. 학습 과정 중에는 정확도를 모니터링해야 합니다.

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

#### Validating your approach

3장에서 배웠듯이, 딥러닝 모델은 훈련 데이터로 직접 평가해서는 안 됩니다. 훈련 과정에서 모델의 정확도를 모니터링하기 위해 "검증 세트"를 사용하는 것이 일반적입니다. 여기서는 원래 훈련 데이터에서 10,000개의 샘플을 추출하여 검증 세트를 만들 것입니다.

그렇다면 왜 테스트 데이터로 모델을 평가하지 않는지 궁금할 수 있습니다. 그게 더 쉬워 보일 수도 있으니까요. 그 이유는 검증 세트에서 얻은 결과를 바탕으로 모델 크기나 에포크 수와 같은 다음 훈련 최적화 방안을 결정해야 하기 때문입니다. 검증 세트에서 얻은 결과는 모델이 검증 데이터에서 더 나은 성능을 보이도록 의도적으로 수정되었기 때문에, 실제 새로운 데이터에서의 모델 성능을 정확하게 반영하지 못하게 됩니다. 따라서 완전히 편향되지 않은 방식으로 최종 평가를 수행할 수 있도록 이전에는 사용하지 않았던 새로운 샘플들을 따로 보관해 두는 것이 좋습니다. 바로 테스트 세트가 그 역할을 합니다. 이에 대해서는 다음 장에서 더 자세히 이야기하겠습니다.

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

이제 512개의 샘플로 구성된 미니 배치로 20 에포크(훈련 데이터의 모든 샘플에 대해 20번의 반복) 동안 모델을 훈련합니다. 동시에, 따로 분리해 둔 10,000개의 샘플에 대한 손실과 정확도를 모니터링합니다. 이를 위해 `model.fit()` 메서드에 `validation_data` 인수를 통해 검증 데이터를 전달합니다.

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val),
)

**`validation_split` 인자 사용**

학습 데이터에서 검증 데이터를 수동으로 분리하여 `validation_data` 인자로 전달하는 대신, `fit()` 함수의 `validation_split` 인자를 사용할 수 있습니다. 이 인자는 학습 데이터의 일부를 검증 데이터로 사용할 수 있도록 지정합니다. 예를 들면 다음과 같습. 데이터로 사용됩니다.

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=512,
    validation_split=0.2,
)

이 예시에서는 `x_train` 및 `y_train` 배열의 샘플 중 20%가 학습에서 제외되어 검증 데이터로 사용됩니다.

CPU 환경에서는 에포크당 2초도 채 걸리지 않으며, 학습은 총 20초 만에 완료됩니다. 매 에포크가 끝날 때마다 모델이 10,000개의 검증 데이터 샘플에 대한 손실과 정확도를 계산하는 동안 잠시 멈춥니다.

모델의 `model.fit()` 호출은 3장에서 살펴본 것처럼 `History` 객체를 반환합니다. 이 객체에는 학습 중에 발생한 모든 정보를 담고 있는 딕셔너리 형태의 `history` 멤버가 있습니다. 이제 `history`를 살펴보겠습니다.

In [ ]:
history_dict = history.history
history_dict.keys()

이 dictionary 에는 훈련 및 검증 중에 모니터링된 각 지표별로 하나씩, 총 네 개의 항목이 있습니다. 다음 두 예제에서는 Matplotlib을 사용하여 훈련 손실과 검증 손실을 나란히 그래프로 나타내고(그림 4.4 참조), 훈련 정확도와 검증 정확도를 그래프로 나타내 보겠습니다(그림 4.5 참조). 모델의 무작위 초기화 방식에 따라 결과가 약간 다를 수 있다는 점에 유의하십시오.

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "r--", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("[IMDB] Training and validation loss")
plt.xlabel("Epochs")
plt.xticks(epochs)
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "r--", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("[IMDB] Training and validation accuracy")
plt.xlabel("Epochs")
plt.xticks(epochs)
plt.ylabel("Accuracy")
plt.legend()
plt.show()

보시다시피, 훈련 손실은 매 에포크마다 감소하고, 훈련 정확도는 매 에포크마다 증가합니다. 이는 경사 하강법 최적화를 실행할 때 예상되는 결과입니다. 최소화하려는 값이 매 반복마다 감소해야 하기 때문입니다. 하지만 검증 손실과 정확도는 그렇지 않습니다. 네 번째 에포크에서 정점을 찍는 것처럼 보입니다. 이는 앞서 경고했던 내용과 일맥상통하는 현상입니다. 훈련 데이터에서 더 나은 성능을 보이는 모델이 처음 접하는 데이터에서도 더 나은 성능을 보일 것이라는 보장은 없습니다. 정확히 말하면, 과적합이 발생한 것입니다. 네 번째 에포크 이후에는 훈련 데이터에 대해 과도하게 최적화되어, 훈련 데이터에 특화된 표현을 학습하게 되고, 훈련 데이터 외의 데이터에는 일반화되지 못하게 됩니다.

이러한 과적합을 방지하려면 네 번째 에포크에서 학습을 중단하는 것이 좋습니다. 일반적으로 과적합을 완화하기 위해 다양한 기법을 사용할 수 있으며, 이는 5장에서 자세히 다루겠습니다.

이제 새로운 모델을 처음부터 4 에포크 동안 학습시킨 후 테스트 데이터로 평가해 보겠습니다.

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

최종 결과는 다음과 같습니다.

In [ ]:
results

이처럼 다소 단순한 접근 방식으로는 88%의 정확도를 얻을 수 있습니다. 최첨단 기술을 활용하면 95%에 가까운 정확도를 달성할 수 있을 것입니다.

#### Using a trained model to generate predictions on new data

모델을 학습시킨 후에는 실제 환경에서 사용해 봐야 합니다. 3장에서 배운 것처럼 predict 메서드를 사용하여 긍정적인 리뷰가 나올 확률을 예측할 수 있습니다.

In [ ]:
model.predict(x_test)

보시다시피, 이 모델은 일부 샘플(0.99 이상 또는 0.01 이하)에 대해서는 높은 신뢰도를 보이지만, 다른 샘플(0.6, 0.4)에 대해서는 신뢰도가 낮습니다.

#### Further experiments (for assignment)

다음 실험들을 통해 여러분이 선택한 아키텍처가 대체로 합리적이라는 것을 확인할 수 있을 것입니다. 물론 개선의 여지는 여전히 있습니다.

* 최종 분류 레이어 전에 표현 레이어를 두 개 사용했습니다. 표현 레이어를 하나 또는 세 개로 줄이고, 검증 및 테스트 정확도에 어떤 영향을 미치는지 살펴보세요.
* 유닛 수를 32개, 64개 등으로 늘려가며 레이어를 구성해 보세요.
* 손실 함수로 이진 교차 엔트로피 대신 평균 제곱 오차를 사용해 보세요.
* 활성화 함수로 ReLU 대신 tanh(신경망 초창기에 널리 사용되었던 활성화 함수)를 사용해 보세요.


#### Wrapping up

이 예제에서 얻을 수 있는 핵심 내용은 다음과 같습니다.

* 일반적으로 원시 데이터를 텐서 형태로 신경망에 입력하려면 상당한 전처리 과정이 필요합니다. 단어 시퀀스는 이진 벡터로 인코딩할 수 있지만, 다른 인코딩 방식도 있습니다.
* ReLU 활성화 함수를 사용하는 Dense 레이어를 여러 개 쌓아 올리면 감정 분류를 포함한 다양한 문제를 해결할 수 있으며, 실제로 자주 사용하게 될 것입니다.
* 이진 분류 문제(두 개의 출력 클래스)에서는 모델의 마지막 레이어가 하나의 유닛과 시그모이드 활성화 함수를 사용하는 Dense 레이어여야 합니다. 모델의 출력은 0에서 1 사이의 스칼라 값으로, 확률을 나타냅니다.
* 이진 분류 문제에서 스칼라 시그모이드 출력을 사용할 경우, 적절한 손실 함수는 binary_crossentropy입니다.
* Adam 옵티마이저는 일반적으로 어떤 문제든 충분히 좋은 성능을 보여줍니다. 따라서 옵티마이저 선택에 대한 부담을 덜 수 있습니다.

* 신경망은 훈련 데이터에 익숙해질수록 과적합되어 결국 이전에 본 적 없는 데이터에서 점점 더 나쁜 결과를 얻게 됩니다. 따라서 훈련 데이터 세트 외의 데이터에 대한 성능을 항상 모니터링해야 합니다!pping up

### Classifying newswires: A multiclass classification example

이전 섹션에서는 밀집 연결 신경망을 사용하여 벡터 입력을 서로 배타적인 두 클래스로 분류하는 방법을 살펴보았습니다. 하지만 클래스가 두 개 이상일 때는 어떻게 해야 할까요?

이 섹션에서는 로이터 뉴스 통신을 46개의 서로 배타적인 주제로 분류하는 모델을 구축합니다. 클래스가 많기 때문에 이 문제는 다중 클래스 분류 문제이며, 각 데이터 포인트는 하나의 범주에만 속해야 하므로, 더 구체적으로는 단일 레이블 다중 클래스 분류 문제입니다. 만약 각 데이터 포인트가 여러 범주(이 경우 주제)에 속할 수 있다면, 다중 레이블 다중 클래스 분류 문제가 될 것입니다.

#### The Reuters dataset

여러분은 1986년 로이터에서 발행한 짧은 뉴스 기사와 그 주제들로 구성된 로이터 데이터셋을 사용하게 됩니다. 이 데이터셋은 텍스트 분류를 위한 간단하고 널리 사용되는 예제 데이터셋입니다. 총 46개의 주제가 있으며, 일부 주제는 다른 주제보다 더 많이 나타나지만, 각 주제는 학습 데이터셋에 최소 10개 이상의 예제를 포함하고 있습니다.

IMDb 및 MNIST 데이터셋과 마찬가지로 로이터 데이터셋도 Keras 패키지에 포함되어 있습니다. 자세히 살펴보겠습니다.

In [ ]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000
)

IMDb 데이터셋과 마찬가지로, `num_words=10000` 인수는 데이터에서 가장 자주 나타나는 10,000개의 단어로 데이터를 제한합니다.

현재 8,982개의 훈련 예제와 2,246개의 테스트 예제가 있습니다.

In [ ]:
len(train_data)

In [ ]:
len(test_data)

IMDb 리뷰와 마찬가지로 각 예시는 정수(단어 인덱스) 목록입니다.

In [ ]:
train_data[10]

궁금해하실 분들을 위해, 이 문자를 단어로 다시 해독하는 방법을 알려드리겠습니다.

In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join(
    # The indices are offset by 3 because 0, 1, and 2 are reserved
    # indices for "padding," "start of sequence," and "unknown."
    [reverse_word_index.get(i - 3, "?") for i in train_data[10]]
)

예제와 연결된 레이블은 0에서 45 사이의 정수, 즉 주제 색인입니다.

In [ ]:
train_labels[10]

#### Preparing the data

이전 예제와 동일한 코드를 사용하여 데이터를 벡터화할 수 있습니다.

In [ ]:
x_train = multi_hot_encode(train_data, num_classes=10000)
x_test = multi_hot_encode(test_data, num_classes=10000)

레이블을 벡터화하는 방법에는 두 가지가 있습니다. 레이블을 정수 형태로 그대로 두거나 원핫 인코딩을 사용하는 것입니다. 원핫 인코딩은 범주형 데이터에 널리 사용되는 형식으로, 범주형 인코딩이라고도 합니다. 이 경우, 레이블의 원핫 인코딩은 각 레이블을 레이블 인덱스 자리에 1을 넣은 모든 요소가 0인 벡터로 임베딩하는 것입니다. 다음은 예시입니다.

In [ ]:
def one_hot_encode(labels, num_classes=46):
    results = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        results[i, label] = 1.0
    return results

y_train = one_hot_encode(train_labels)
y_test = one_hot_encode(test_labels)

참고로 Keras에는 이를 수행하는 내장 기능이 있습니다.

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

#### Building your model

이 주제 분류 문제는 이전의 영화 리뷰 분류 문제와 유사해 보입니다. 두 경우 모두 짧은 텍스트 조각을 분류하는 것이 목표입니다. 하지만 여기에는 새로운 제약 조건이 있습니다. 출력 클래스의 수가 2개에서 46개로 늘어난 것입니다. 출력 공간의 차원이 훨씬 커졌습니다.

이전 예제에서 사용했던 것과 같은 Dense 레이어 스택에서는 각 레이어가 이전 레이어의 출력에 있는 정보에만 접근할 수 있습니다. 한 레이어에서 분류 문제와 관련된 정보가 손실되면 이후 레이어에서 해당 정보를 복구할 수 없습니다. 즉, 각 레이어가 정보 병목 현상을 일으킬 수 있습니다. 이전 예제에서는 16차원의 중간 레이어를 사용했지만, 16차원 공간으로는 46개의 서로 다른 클래스를 구분하는 학습을 하기에 충분하지 않을 수 있습니다. 이러한 작은 레이어는 정보 병목 현상을 일으켜 관련 정보를 영구적으로 손실시킬 수 있습니다.

따라서 더 큰 중간 레이어를 사용해야 합니다. 64개의 유닛을 사용하겠습니다.

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(46, activation="softmax"),
    ]
)

이 아키텍처에 대해 주목해야 할 두 가지 사항이 더 있습니다.

마지막 레이어는 크기가 46인 Dense 레이어입니다. 즉, 각 입력 샘플에 대해 네트워크는 46차원 벡터를 출력합니다. 이 벡터의 각 요소(각 차원)는 서로 다른 출력 클래스를 나타냅니다.
마지막 레이어는 소프트맥스 활성화 함수를 사용합니다. MNIST 예제에서 보셨듯이, 모델은 46개의 서로 다른 출력 클래스에 대한 확률 분포를 출력합니다. 모든 입력 샘플에 대해 모델은 46차원 출력 벡터를 생성하며, 여기서 output[i]는 샘플이 클래스 i에 속할 확률입니다. 46개의 점수 합은 모두 1이 됩니다.
이 경우에 가장 적합한 손실 함수는 범주형 교차 엔트로피(categorical_crossentropy)입니다. 이는 두 확률 분포 사이의 거리를 측정합니다. 여기서는 모델이 출력하는 확률 분포와 실제 레이블 분포 사이의 거리를 측정합니다. 이 두 분포 사이의 거리를 최소화함으로써 모델은 실제 레이블에 최대한 가까운 값을 출력하도록 학습됩니다.

지난번과 마찬가지로 정확도도 모니터링할 것입니다. 하지만 이 경우 정확도는 다소 투박한 지표입니다. 예를 들어, 모델이 주어진 샘플에 대해 첫 번째 선택이 틀렸더라도 두 번째 선택에서 올바른 클래스를 선택했다면, 해당 샘플에 대한 정확도는 0이 됩니다. 비록 그런 모델이라도 무작위로 예측하는 것보다는 훨씬 나은 결과를 보여주겠지만 말입니다. 이러한 경우에 더 적합한 지표는 상위 k개 정확도, 예를 들어 상위 3개 정확도 또는 상위 5개 정확도입니다. 이는 모델의 상위 k개 예측 중 올바른 클래스가 포함되었는지 여부를 측정합니다. 이제 우리 모델에 상위 3개 정확도를 추가해 보겠습니다.

In [ ]:
top_3_accuracy = keras.metrics.TopKCategoricalAccuracy(
    k=3, name="top_3_accuracy"
)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy", top_3_accuracy],
)

#### Validating your approach

훈련 데이터에서 1,000개의 샘플을 따로 떼어내어 검증 세트로 사용하겠습니다.

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

이제 모델을 20 에포크 동안 학습시켜 보겠습니다.

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val),
)

마지막으로 손실 및 정확도 곡선을 표시해 보겠습니다(그림 4.6 및 4.7 참조).

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "r--", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.xticks(epochs)
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "r--", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.xticks(epochs)
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history["top_3_accuracy"]
val_acc = history.history["val_top_3_accuracy"]
plt.plot(epochs, acc, "r--", label="Training top-3 accuracy")
plt.plot(epochs, val_acc, "b", label="Validation top-3 accuracy")
plt.title("Training and validation top-3 accuracy")
plt.xlabel("Epochs")
plt.xticks(epochs)
plt.ylabel("Top-3 accuracy")
plt.legend()
plt.show()

해당 모델은 9번의 에포크 이후 과적합되기 시작합니다. 새로운 모델을 처음부터 다시 학습시켜 9번의 에포크를 거친 후 테스트 세트에서 평가해 보겠습니다.

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(46, activation="softmax"),
    ]
)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    x_train,
    y_train,
    epochs=9,
    batch_size=512,
)
results = model.evaluate(x_test, y_test)

최종 결과는 다음과 같습니다.

In [ ]:
results

이 접근 방식은 약 80%의 정확도를 달성합니다. 균형 잡힌 이진 분류 문제의 경우, 순수하게 무작위로 분류기를 사용하면 50%의 정확도를 얻을 수 있습니다. 하지만 이 경우에는 46개의 클래스가 있으며, 각 클래스가 균등하게 분포되어 있지 않을 수 있습니다. 그렇다면 무작위로 분류기를 선택했을 때의 정확도는 얼마나 될까요? 이를 확인하기 위해 간단한 무작위 분류기를 구현해 볼 수 있습니다.

In [ ]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels == test_labels_copy)
hits_array.mean()

보시다시피, 무작위 분류기는 약 19%의 분류 정확도를 보일 것이므로, 이러한 관점에서 우리 모델의 결과는 상당히 좋아 보입니다.

#### Generating predictions on new data

새로운 샘플에 대해 모델의 predict 메서드를 호출하면 각 샘플에 대해 46개 주제 전체에 대한 클래스 확률 분포가 반환됩니다. 이제 모든 테스트 데이터에 대한 주제 예측을 생성해 보겠습니다.

In [ ]:
predictions = model.predict(x_test)

"예측"의 각 항목은 길이가 46인 벡터입니다.


In [ ]:
predictions[0].shape

이 벡터의 계수들은 확률 분포를 형성하므로 합이 1이 됩니다.

In [ ]:
np.sum(predictions[0])

가장 큰 항목은 예측된 클래스, 즉 가장 높은 확률을 가진 클래스입니다.

In [ ]:
np.argmax(predictions[0])

#### A different way to handle the labels and the loss

앞서 언급했듯이 레이블을 인코딩하는 또 다른 방법은 다음과 같이 정수 텐서 형태로 그대로 두는 것입니다.

In [ ]:
y_train = train_labels
y_test = test_labels

이 접근 방식에서 달라지는 유일한 점은 손실 함수의 선택입니다. 목록 4.22에서 사용된 손실 함수인 categorical_crossentropy는 레이블이 범주형 인코딩을 따른다고 가정합니다. 정수 레이블을 사용하는 경우에는 sparse_categorical_crossentropy를 사용해야 합니다.

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

이 새로운 손실 함수는 수학적으로는 범주형 교차 엔트로피와 동일하지만 인터페이스가 다릅니다.

#### The importance of having sufficiently large intermediate layers

앞서 최종 출력값이 46차원이므로 46개보다 훨씬 적은 유닛을 가진 중간 계층은 피해야 한다고 언급했습니다. 이제 46차원보다 훨씬 작은, 예를 들어 4차원의 중간 계층을 사용하여 정보 병목 현상을 발생시키면 어떤 일이 발생하는지 살펴보겠습니다.

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(4, activation="relu"),
        layers.Dense(46, activation="softmax"),
    ]
)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=128,
    validation_data=(x_val, y_val),
)

현재 모델의 검증 정확도는 약 71%로, 8%p 감소했습니다. 이러한 감소는 주로 46개 클래스의 분리 초평면을 복원하는 데 필요한 많은 정보를 너무 낮은 차원의 중간 공간에 압축하려 했기 때문입니다. 모델은 필요한 정보의 대부분을 4차원 표현에 담을 수 있지만, 모든 정보를 담지는 못합니다.

#### Further experiments

이전 예시와 마찬가지로, 이러한 모델을 사용할 때 어떤 설정을 해야 하는지 직관적으로 이해할 수 있도록 다음과 같은 실험을 해보시기 바랍니다.

레이어 크기를 32개, 128개 등으로 늘려가며 시도해 보세요.

이전에는 최종 소프트맥스 분류 레이어 앞에 두 개의 중간 레이어를 사용했지만, 이번에는 하나의 중간 레이어 또는 세 개의 중간 레이어를 사용해 보세요.

#### Wrapping up

이 예시에서 얻을 수 있는 핵심 내용은 다음과 같습니다.

* N개의 클래스로 데이터 포인트를 분류하려는 경우, 모델은 크기가 N인 Dense 레이어로 끝나야 합니다.
* 단일 레이블 다중 클래스 분류 문제에서는, N개의 출력 클래스에 대한 확률 분포를 출력하기 위해 모델의 마지막 레이어는 소프트맥스 활성화 함수여야 합니다.
* 이러한 문제에는 범주형 교차 엔트로피(categorical crossentropy) 손실 함수를 사용하는 것이 거의 항상 적합합니다. 이 함수는 모델이 출력하는 확률 분포와 목표 변수의 실제 분포 사이의 거리를 최소화합니다.
* 다중 클래스 분류에서 레이블을 처리하는 방법은 두 가지가 있습니다.
  - 범주형 인코딩(원핫 인코딩이라고도 함)을 통해 레이블을 인코딩하고 categorical_crossentropy 손실 함수를 사용하는 방법
  - 레이블을 정수로 인코딩하고 sparse_categorical_crossentropy 손실 함수를 사용하는 방법
* 많은 수의 범주로 데이터를 분류해야 하는 경우, 너무 작은 중간 레이어로 인해 모델에 정보 병목 현상이 발생하지 않도록 해야 합니다.

### Predicting house prices: A regression example

앞의 두 예시는 입력 데이터 포인트의 단일 이산 레이블을 예측하는 것이 목표인 분류 문제로 간주되었습니다. 머신 러닝 문제의 또 다른 일반적인 유형은 회귀 문제인데, 이는 이산 레이블 대신 연속적인 값을 예측하는 것입니다. 예를 들어 기상 데이터를 기반으로 내일의 기온을 예측하거나 소프트웨어 프로젝트의 사양을 기반으로 완료하는 데 걸리는 시간을 예측하는 것이 이에 해당합니다.

>혼동하기 쉽지만, 로지스틱 회귀는 회귀 알고리즘이 아니라 분류 알고리즘입니다.

#### The California Housing Price dataset

1990년 인구 조사 데이터를 기반으로 캘리포니아 여러 지역의 주택 중간 가격을 예측해 보겠습니다.

데이터 세트의 각 데이터 포인트는 동일한 지역에 위치한 주택 그룹인 "블록 그룹"에 대한 정보를 나타냅니다. 이를 구역(district)으로 생각할 수 있습니다. 이 데이터 세트는 600개의 구역으로 구성된 "소형" 버전과 20,640개의 구역으로 구성된 "대형" 버전 두 가지가 있습니다. 실제 데이터 세트는 종종 매우 작을 수 있으므로 소형 버전을 사용하겠습니다. 이러한 경우를 처리하는 방법을 알아야 합니다.

각 구역에 대해 다음 정보를 알 수 있습니다.

* 해당 지역의 대략적인 지리적 중심의 경도와 위도
* 해당 구역의 주택 중간 연령
* 해당 구역의 인구 (구역 규모는 비교적 작으며 평균 인구는 1,425.5명입니다.)
* 총 가구 수
* 해당 가구의 중간 소득
* 해당 구역에 있는 모든 주택의 총 방 개수 일반적으로 이 수치는 수천 단위입니다.
* 해당 지역의 총 침실 수입니다.

총 8개의 변수가 있습니다(경도와 위도는 두 개의 변수로 계산). 목표는 이러한 변수를 사용하여 해당 지역 주택의 중간값을 예측하는 것입니다. 이제 데이터를 불러오면서 시작해 보겠습니다.

In [ ]:
from keras.datasets import california_housing

(train_data, train_targets), (test_data, test_targets) = (
    california_housing.load_data(version="small")
)

데이터를 살펴보겠습니다.

In [ ]:
train_data.shape

In [ ]:
test_data.shape

보시다시피, 우리는 480개의 훈련 샘플과 120개의 테스트 샘플을 가지고 있으며, 각 샘플은 8개의 수치형 특징을 포함합니다. 목표는 해당 지역의 주택 중간 가격(달러)입니다.

In [ ]:
train_targets

가격은 6만 달러에서 50만 달러 사이입니다. 싸게 들릴 수도 있지만, 이 가격은 1990년 당시 가격이며 물가상승률을 반영하지 않았다는 점을 기억하세요.

#### Preparing the data

범위가 매우 다양한 값을 신경망에 입력하는 것은 문제가 될 수 있습니다. 모델이 이러한 이질적인 데이터에 자동으로 적응할 수는 있겠지만, 학습 과정은 분명히 어려워질 것입니다. 이러한 데이터를 처리하는 널리 사용되는 모범 사례는 특징별 정규화입니다. 입력 데이터의 각 특징(입력 데이터 행렬의 열)에 대해 해당 특징의 평균을 빼고 표준 편차로 나누어 특징이 0을 중심으로 하고 표준 편차가 1이 되도록 하는 것입니다. 이는 NumPy를 사용하여 쉽게 구현할 수 있습니다.

In [ ]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
x_train = (train_data - mean) / std
x_test = (test_data - mean) / std

테스트 데이터 정규화에 사용되는 값은 훈련 데이터를 기반으로 계산된다는 점에 유의하세요. 테스트 데이터를 기반으로 계산된 값은 데이터 정규화와 같은 간단한 작업이라도 워크플로우에서 절대 사용해서는 안 됩니다.

또한, 목표값도 조정해야 합니다. 정규화된 입력값은 0에 가까운 작은 범위에 있으며, 모델의 가중치는 작은 난수로 초기화됩니다. 따라서 모델 학습 초기에는 예측값 또한 작은 값을 갖게 됩니다. 목표값이 60,000~500,000 범위에 있다면, 모델은 이 값을 출력하기 위해 매우 큰 가중치가 필요합니다. 학습률이 작으면 목표값에 도달하는 데 매우 오랜 시간이 걸립니다. 가장 간단한 해결책은 모든 목표값을 100,000으로 나누어 가장 작은 목표값을 0.6, 가장 큰 목표값을 5로 만드는 것입니다. 그런 다음 모델의 예측값에 100,000을 곱하여 다시 달러 값으로 변환할 수 있습니다.

In [ ]:
y_train = train_targets / 100000
y_test = test_targets / 100000

#### Building your model

사용 가능한 샘플 수가 매우 적기 때문에, 각각 64개의 유닛으로 구성된 두 개의 중간 레이어를 가진 매우 작은 모델을 사용하게 됩니다. 일반적으로 훈련 데이터가 적을수록 과적합이 심해지는데, 작은 모델을 사용하는 것은 과적합을 완화하는 한 가지 방법입니다.

In [ ]:
def get_model():
    # Because you need to instantiate the same model multiple times,
    # you use a function to construct it.
    model = keras.Sequential(
        [
            layers.Dense(64, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(1),
        ]
    )
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

모델은 활성화 함수가 없는 단일 유닛으로 끝납니다. 즉, 선형 레이어가 됩니다. 이는 스칼라 회귀, 즉 단일 연속 값을 예측하려는 회귀 문제에서 흔히 볼 수 있는 구성입니다. 활성화 함수를 적용하면 출력값이 가질 수 있는 범위가 제한됩니다. 예를 들어, 마지막 레이어에 시그모이드 활성화 함수를 적용하면 모델은 0에서 1 사이의 값만 예측하도록 학습할 수 있습니다. 하지만 여기서는 마지막 레이어가 순수 선형 레이어이므로 모델은 어떤 범위의 값이라도 예측하도록 자유롭게 학습할 수 있습니다.

모델을 컴파일할 때 평균 제곱 오차(mean_squared_error) 손실 함수를 사용합니다. 평균 제곱 오차는 예측값과 목표값의 차이를 제곱한 값입니다. 이는 회귀 문제에 널리 사용되는 손실 함수입니다.

또한 학습 과정에서 평균 절대 오차(MAE)라는 새로운 지표를 모니터링합니다. MAE는 예측값과 목표값의 차이의 절댓값입니다. 예를 들어, 이 문제에서 MAE가 0.5라는 것은 예측값이 평균적으로 5만 달러 정도 오차가 있다는 것을 의미합니다(목표 스케일링 계수가 10만이라는 점을 기억하세요).

#### Validating your approach using K-fold validation

모델의 매개변수(예: 학습에 사용되는 에포크 수)를 조정하면서 모델을 평가하려면 이전 예제에서처럼 데이터를 학습 세트와 검증 세트로 나눌 수 있습니다. 하지만 데이터 포인트 수가 적으면 검증 세트의 크기가 매우 작아집니다(예: 약 100개). 결과적으로, 검증에 사용할 데이터 포인트와 학습에 사용할 데이터 포인트를 선택하는 방식에 따라 검증 점수가 크게 달라질 수 있습니다. 즉, 검증 분할에 따라 검증 점수의 분산이 커질 수 있습니다. 이는 모델을 안정적으로 평가하는 것을 어렵게 만듭니다.

이러한 상황에서 가장 좋은 방법은 K-겹 교차 검증(그림 4.9 참조)을 사용하는 것입니다. K-겹 교차 검증은 사용 가능한 데이터를 K개의 파티션(일반적으로 K=4 또는 5)으로 분할하고, K개의 동일한 모델을 생성한 다음, 각 모델을 K-1개의 파티션에서 학습시키고 나머지 하나의 파티션에서 평가하는 방식으로 구성됩니다. 사용된 모델의 검증 점수는 얻은 K개의 검증 점수의 평균입니다. 코드 측면에서 이는 매우 간단합니다.

<img src="https://deeplearningwithpython.io/images/ch04/3-fold-cross-validation.40bb5356.png" width="600"><p style="text-align:center">Figure 4.9: Three-fold cross-validation</p>다.

In [ ]:
k = 4
num_val_samples = len(x_train) // k
num_epochs = 50
all_scores = []
for i in range(k):
    print(f"Processing fold #{i + 1}")
    fold_x_val = x_train[i * num_val_samples : (i + 1) * num_val_samples]
    fold_y_val = y_train[i * num_val_samples : (i + 1) * num_val_samples]
    fold_x_train = np.concatenate(
        [x_train[: i * num_val_samples], x_train[(i + 1) * num_val_samples :]],
        axis=0,
    )
    fold_y_train = np.concatenate(
        [y_train[: i * num_val_samples], y_train[(i + 1) * num_val_samples :]],
        axis=0,
    )
    model = get_model()
    model.fit(
        fold_x_train,
        fold_y_train,
        epochs=num_epochs,
        batch_size=16,
        verbose=0,
    )
    scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
    val_loss, val_mae = scores
    all_scores.append(val_mae)

num_epochs = 50으로 실행하면 다음과 같은 결과가 나옵니다.

In [ ]:
[round(value, 3) for value in all_scores]

In [ ]:
round(np.mean(all_scores), 3)

실제로 여러 번의 실행 결과는 0.232에서 0.349까지 의미 있는 차이를 보이는 검증 점수를 나타냅니다. 평균값(0.296)은 단일 점수보다 훨씬 더 신뢰할 수 있는 지표이며, 이것이 바로 K-겹 교차 검증의 핵심입니다. 이 경우 평균적으로 29,600달러의 오차가 발생하는데, 가격 범위가 60,000달러에서 500,000달러라는 점을 고려하면 상당한 차이입니다.

이제 모델 학습 시간을 200 에포크로 늘려 보겠습니다. 각 에포크에서 모델의 성능을 기록하기 위해 학습 루프를 수정하여 에포크별 검증 점수를 로그로 저장하도록 하겠습니다.

In [ ]:
k = 4
num_val_samples = len(x_train) // k
num_epochs = 200
all_mae_histories = []
for i in range(k):
    print(f"Processing fold #{i + 1}")
    fold_x_val = x_train[i * num_val_samples : (i + 1) * num_val_samples]
    fold_y_val = y_train[i * num_val_samples : (i + 1) * num_val_samples]
    fold_x_train = np.concatenate(
        [x_train[: i * num_val_samples], x_train[(i + 1) * num_val_samples :]],
        axis=0,
    )
    fold_y_train = np.concatenate(
        [y_train[: i * num_val_samples], y_train[(i + 1) * num_val_samples :]],
        axis=0,
    )
    model = get_model()
    history = model.fit(
        fold_x_train,
        fold_y_train,
        validation_data=(fold_x_val, fold_y_val),
        epochs=num_epochs,
        batch_size=16,
        verbose=0,
    )
    mae_history = history.history["val_mean_absolute_error"]
    all_mae_histories.append(mae_history)

그런 다음 모든 폴드에 대한 에포크별 평균 절대 오차(MAE) 점수의 평균을 계산할 수 있습니다.

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)
]

In [ ]:
epochs = range(1, len(average_mae_history) + 1)
plt.plot(epochs, average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

그래프의 스케일 문제 때문에 그래프를 읽기가 다소 어려울 수 있습니다. 처음 몇 에포크 동안의 검증 MAE가 이후 값들보다 훨씬 높게 나타나기 때문입니다. 따라서 나머지 곡선과 스케일이 다른 처음 10개의 데이터 포인트는 제외하겠습니다.

In [ ]:
truncated_mae_history = average_mae_history[10:]
epochs = range(10, len(truncated_mae_history) + 10)
plt.plot(epochs, truncated_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

이 그래프(그림 4.11 참조)에 따르면, 검증 MAE는 120~140 에포크(이 수치에는 생략된 10 에포크가 포함됨) 이후에는 크게 개선되지 않습니다. 그 시점을 넘어서면 과적합이 시작됩니다.

모델의 다른 매개변수(에포크 수 외에도 중간 레이어의 크기 조정 등) 튜닝을 완료한 후에는 최적의 매개변수를 사용하여 모든 훈련 데이터로 최종 프로덕션 모델을 학습시키고 테스트 데이터에서 성능을 평가할 수 있습니다.

In [ ]:
model = get_model()
model.fit(x_train, y_train, epochs=130, batch_size=16, verbose=0)
test_mean_squared_error, test_mean_absolute_error = model.evaluate(
    x_test, y_test
)

In [ ]:
round(test_mean_absolute_error, 3)

우리는 여전히 평균적으로 약 3만 1천 달러 정도 차이가 납니다.

#### Generating predictions on new data

이진 분류 모델에서 predict() 함수를 호출하면 각 입력 샘플에 대해 0에서 1 사이의 스칼라 값이 반환됩니다. 다중 클래스 분류 모델에서는 각 샘플에 대해 모든 클래스에 대한 확률 분포가 반환됩니다. 이제 이 스칼라 회귀 모델을 사용하면 predict() 함수는 샘플의 가격을 수십만 달러 단위로 예측한 값을 반환합니다.

In [ ]:
predictions = model.predict(x_test)
predictions[0]

테스트 대상 지역 중 첫 번째 지역의 평균 주택 가격은 약 28만 3천 달러로 예측됩니다.

#### Wrapping up

이 스칼라 회귀 예제에서 얻을 수 있는 핵심 내용은 다음과 같습니다.

* 회귀 분석에는 분류에 사용했던 손실 함수와는 다른 손실 함수를 사용합니다. 평균 제곱 오차(MSE)는 회귀 분석에 흔히 사용되는 손실 함수입니다.
* 마찬가지로 회귀 분석에 사용되는 평가 지표도 분류에 사용되는 지표와 다릅니다. 당연히 정확도라는 개념은 회귀 분석에는 적용되지 않습니다. 회귀 분석에서 흔히 사용되는 평가 지표는 평균 절대 오차(MAE)입니다.
* 입력 데이터의 특징들이 서로 다른 범위의 값을 가질 경우, 전처리 단계에서 각 특징을 독립적으로 스케일링해야 합니다.
* 사용 가능한 데이터가 적을 때는 K-겹 교차 검증을 통해 모델을 안정적으로 평가할 수 있습니다.
* 훈련 데이터가 부족할 때는 심각한 과적합을 방지하기 위해 중간 레이어가 적은(일반적으로 한두 개) 작은 모델을 사용하는 것이 좋습니다.